# Neo4j에 그래프 로드

In [ ]:
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [ ]:
import pickle

with open('samples/graph_docs.pkl', 'rb') as f:
    loaded_graph_docs = pickle.load(f)

In [ ]:
from langchain.graphs import Neo4jGraph
from py2neo import Graph

py_graph = Graph()
n4j_graph = Neo4jGraph()
vector_search_enabled = True
index_name = "content_embedding_index"

def init_graph(vector_search_enabled: bool):
    py_graph.run("MATCH (n) DETACH DELETE n")

    if vector_search_enabled:
        query = """
            CREATE VECTOR INDEX $index_name IF NOT EXISTS
            FOR (n:Content) ON (n.embedding)
            OPTIONS {
                indexConfig: {
                    `vector.dimensions`: 1024,
                    `vector.similarity_function`: 'cosine'
                }
            }
        """
        params = {"index_name": index_name}
        py_graph.run(query, params)

    py_graph.run("""
        CREATE FULLTEXT INDEX Search_Content_by_FullText IF NOT EXISTS
        FOR (n:Content) 
        ON EACH [n.text] 
        OPTIONS { indexConfig: { `fulltext.analyzer`: "english"}}
        """)

init_graph(vector_search_enabled)

In [ ]:
n4j_graph.add_graph_documents(
  loaded_graph_docs, 
  baseEntityLabel=True
)

### Vector Search (Global)

In [ ]:
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import Neo4jVector

if vector_search_enabled == True:
    embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name="content_embedding_index",
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )

    question = "Bedrock에서 KnowledgeBase 성능을 최적화 하는 방법"
    question_embedding = embeddings.embed_query(question)
    k=5

    cypher_query = f"""
        CALL db.index.vector.queryNodes('{index_name}', $k, $question_embedding) YIELD node, score
        WHERE node:Content
        RETURN node.text AS text, score, {{ }} AS metadata
        ORDER BY score DESC
    """

    params = {
        "question_embedding": question_embedding,
        "k": k
    }

    results = n4j_graph.query(cypher_query, params=params)

    for result in results:
        print("=================\n")
        print(result['text'])
        print("=================\n")

### Vector Search (Local)

In [ ]:
question = "Bedrock에서 Function Calling 활용하는 방법"
parent_id = 1606
k=5

if vector_search_enabled == True:
    embeddings = BedrockEmbeddings(model_id="cohere.embed-multilingual-v3", region_name="us-east-1")

    question_embedding = embeddings.embed_query(question)

    cypher_query = """
        MATCH (parent)-[:HAS_CONTENTS]->(child:Content)
        WHERE id(parent) = $parent_id

        CALL db.index.vector.queryNodes($index_name, $k, $question_embedding) YIELD node, score
        WHERE node = child

        RETURN node.text AS text, score, {} AS metadata
        ORDER BY score DESC
        LIMIT $k
    """

    params = {
        "parent_id": parent_id,
        "question_embedding": question_embedding,
        "k": k,
        "index_name": index_name
    }

    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        index_name=index_name,
        node_label="Content",
        text_node_property="text", 
        embedding_node_property="embedding"
    )

    results = vector_store.query(cypher_query, params=params)

    for result in results:
        print("=================\n")
        print(result['text'])
        print("=================\n")

